# 🔬 Quantum-Enhanced Credit Risk Prediction

**Framework**: Quantum-Enhanced Smart Computing Framework for Sustainable Credit Risk Decision Communication  
**Reference**: Jain, Singh, et al. (2026)  

This notebook provides a comprehensive, interactive walkthrough of the hybrid quantum-classical credit risk prediction pipeline.

---

## Table of Contents
1. [Setup & Imports](#1-setup--imports)
2. [Data Loading & Exploration](#2-data-loading--exploration)
3. [Preprocessing Pipeline (Eq. 1–3)](#3-preprocessing-pipeline)
4. [Spectral Feature Engineering (Eq. 5–6)](#4-spectral-feature-engineering)
5. [Quantum Circuit Visualisation (Eq. 4, 7)](#5-quantum-circuit-visualisation)
6. [Quantum Kernel & QSVM (Eq. 8)](#6-quantum-kernel--qsvm)
7. [Classical Baselines](#7-classical-baselines)
8. [Results & Comparison](#8-results--comparison)

## 1. Setup & Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.preprocessing import (
    generate_synthetic_credit_data,
    mean_imputation,
    min_max_normalize,
    one_hot_encode,
    preprocess_pipeline,
)
from src.fft_dct_features import apply_fft, apply_dct, extract_spectral_features
from src.quantum_kernel import QuantumKernel, angle_embedding, variational_circuit
from src.svm_classifier import QuantumSVM
from src.classical_baselines import evaluate_all_baselines
from src.evaluation import compute_metrics, plot_confusion_matrix, plot_roc_curve, plot_model_comparison

sns.set_theme(style='whitegrid', palette='deep')
%matplotlib inline

print('All imports successful ✓')

## 2. Data Loading & Exploration

In [ ]:
# Generate synthetic credit risk data (or load your own CSV)
df_raw = generate_synthetic_credit_data(n_samples=2000, random_state=42)
print(f'Dataset shape: {df_raw.shape}')
df_raw.head()

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df_raw['loan_status'].value_counts().plot.bar(ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Loan Status Distribution')
axes[0].set_xticklabels(['Low Risk (0)', 'High Risk (1)'], rotation=0)

df_raw.select_dtypes(include=[np.number]).hist(ax=axes[1] if False else None, figsize=(14, 8), bins=30, edgecolor='white')
plt.suptitle('Numerical Feature Distributions', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Missing values
missing = df_raw.isnull().sum()
print('Missing values per column:')
print(missing[missing > 0])

## 3. Preprocessing Pipeline

Implementing Eq. 1 (Mean Imputation), Eq. 2 (Min-Max Normalisation), and Eq. 3 (One-Hot Encoding).

In [ ]:
# Step-by-step preprocessing
df_imputed = mean_imputation(df_raw)
print(f'After imputation — missing values: {df_imputed.isnull().sum().sum()}')

df_encoded = one_hot_encode(df_imputed)
print(f'After one-hot encoding — columns: {df_encoded.shape[1]}')

target = df_encoded['loan_status']
features_df = df_encoded.drop(columns=['loan_status'])
features_df = min_max_normalize(features_df)
print(f'After normalisation — range: [{features_df.min().min():.4f}, {features_df.max().max():.4f}]')

In [ ]:
# Or use the full pipeline in one call
X_train, X_test, y_train, y_test, feature_names = preprocess_pipeline(random_state=42)
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')

## 4. Spectral Feature Engineering

**Eq. 5**: FFT Power Spectrum — captures frequency-domain energy distribution  
**Eq. 6**: DCT-II Energy Compaction — concentrates signal energy in few coefficients

In [ ]:
# Demonstrate FFT on a synthetic signal
t = np.linspace(0, 1, 256, endpoint=False)
signal = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 20 * t)

power_spectrum = apply_fft(signal)
dct_coeffs = apply_dct(signal, n_components=20)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(t, signal, color='#3498db')
axes[0].set_title('Input Signal')
axes[0].set_xlabel('Time')

axes[1].stem(power_spectrum[:50], linefmt='#e74c3c', markerfmt='ro', basefmt='gray')
axes[1].set_title('FFT Power Spectrum (Eq. 5)')
axes[1].set_xlabel('Frequency Index')

axes[2].bar(range(len(dct_coeffs)), np.abs(dct_coeffs), color='#2ecc71')
axes[2].set_title('DCT Coefficients (Eq. 6)')
axes[2].set_xlabel('Component Index')

plt.tight_layout()
plt.show()

In [ ]:
# Apply spectral features to the credit data
df_train = pd.DataFrame(X_train, columns=feature_names)
spectral_cols = feature_names[:3]  # first 3 numeric features
df_spectral = extract_spectral_features(df_train, columns=spectral_cols, n_dct_components=3)

new_cols = [c for c in df_spectral.columns if c not in feature_names]
print(f'New spectral features ({len(new_cols)}): {new_cols}')

## 5. Quantum Circuit Visualisation

**Eq. 4**: Angle Embedding — maps features to qubit rotations  
**Eq. 7**: Variational Circuit — R_X/R_Z rotations + CNOT entanglers

In [ ]:
import pennylane as qml

N_QUBITS_VIZ = 4
N_LAYERS_VIZ = 1

dev = qml.device('default.qubit', wires=N_QUBITS_VIZ)
wires = list(range(N_QUBITS_VIZ))

@qml.qnode(dev)
def demo_circuit(x, params):
    angle_embedding(x, wires)
    variational_circuit(params, wires, n_layers=N_LAYERS_VIZ)
    return qml.probs(wires=wires)

x_demo = np.array([0.2, 0.5, 0.8, 0.3])
params_demo = np.random.default_rng(42).uniform(0, 2*np.pi, (N_LAYERS_VIZ, N_QUBITS_VIZ, 2))

# Draw the circuit
fig, ax = qml.draw_mpl(demo_circuit)(x_demo, params_demo)
plt.title('Quantum Feature Map Circuit (Eq. 4 + Eq. 7)', fontsize=12)
plt.tight_layout()
plt.show()

## 6. Quantum Kernel & QSVM

**Eq. 8**: $K(x_i, x_j) = |\langle\phi(x_i)|\phi(x_j)\rangle|^2$

We use subset sampling for tractable quantum simulation on local hardware.

In [ ]:
# Subset for quantum tractability
SAMPLE_SIZE = 50
N_Q = 4

rng = np.random.default_rng(42)
idx_train = rng.choice(len(X_train), SAMPLE_SIZE, replace=False)
idx_test = rng.choice(len(X_test), SAMPLE_SIZE // 4, replace=False)

X_train_q = X_train[idx_train, :N_Q]
y_train_q = y_train[idx_train]
X_test_q = X_test[idx_test, :N_Q]
y_test_q = y_test[idx_test]

print(f'Quantum subset: train={len(X_train_q)}, test={len(X_test_q)}, features={N_Q}')

In [ ]:
# Compute quantum kernel matrix
qk = QuantumKernel(n_qubits=N_Q, n_layers=1, random_state=42)
K_train = qk.compute_kernel_matrix(X_train_q, verbose=True)

# Visualise kernel matrix
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(K_train, cmap='viridis', ax=ax, vmin=0, vmax=1)
ax.set_title(f'Quantum Kernel Matrix ({SAMPLE_SIZE}×{SAMPLE_SIZE})', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Train Quantum SVM
qsvm = QuantumSVM(kernel_fn=qk, n_qubits=N_Q, n_layers=1)
qsvm.fit(X_train_q, y_train_q, verbose=True)

qsvm_preds = qsvm.predict(X_test_q)
qsvm_metrics = compute_metrics(y_test_q, qsvm_preds)
print(f'\nQSVM Accuracy: {qsvm_metrics["accuracy"]:.4f}')
print(f'QSVM F1-Score: {qsvm_metrics["f1"]:.4f}')

## 7. Classical Baselines

In [ ]:
baseline_results = evaluate_all_baselines(
    X_train, X_test, y_train, y_test,
    cnn_epochs=30,
    verbose=True
)

## 8. Results & Comparison

In [ ]:
all_metrics = {}
all_metrics['Quantum SVM'] = qsvm_metrics

for name, res in baseline_results.items():
    m = compute_metrics(y_test, res['predictions'], res['probabilities'])
    all_metrics[name] = m

# Display as table
results_df = pd.DataFrame(all_metrics).T
results_df = results_df.sort_values('accuracy', ascending=False)
results_df.style.format('{:.4f}').background_gradient(cmap='Greens', axis=0)

In [ ]:
# Model comparison bar chart
plot_model_comparison(all_metrics)
plt.show()

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(1, len(baseline_results) + 1, figsize=(5 * (len(baseline_results) + 1), 4))

from sklearn.metrics import confusion_matrix as cm_fn

# QSVM
cm = cm_fn(y_test_q, qsvm_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Quantum SVM')

# Baselines
for idx, (name, res) in enumerate(baseline_results.items(), start=1):
    cm = cm_fn(y_test, res['predictions'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(name)

plt.suptitle('Confusion Matrices — All Models', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

**Summary**: This notebook demonstrated the complete hybrid quantum-classical credit risk prediction pipeline,
from data preprocessing through spectral feature engineering to quantum kernel computation and model comparison.

The Quantum SVM leverages an exponentially large Hilbert space (2ⁿ dimensions) to capture feature correlations
intractable for classical kernels, achieving competitive performance on credit risk classification.